In [6]:
from mesa import Model
from mesa import Agent
from mesa.space import MultiGrid
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector
import random
import numpy as np
import pickle

from a_star_pathfinding import a_star
from vision import lines
from vision import transform

In [7]:
def euclidean(pos_a, pos_b):
    return ((pos_a[0] - pos_b[0])**2 + (pos_a[1] - pos_b[1])**2)**.5

In [136]:
class AmongUs(Model):
    def __init__(self, map_name, n_crew, n_impo):
        super().__init__()

        # generate grid
        self.height = 138
        self.width = 242
        self.grid = MultiGrid(self.width, self.height, torus=True)

        # generate map
        self.generate_map(map_name)
        
        # load in vision dictionary
        vision_file = open("vision_dict.pkl", "rb")
        vision_dict = pickle.load(vision_file)
        
        # create schedulers for agents
        self.schedule_Crewmate = RandomActivation(self)
        self.schedule_Imposter = RandomActivation(self)
        
        # initialize agents
        self.n_crew = n_crew
        self.n_impo = n_impo
        self.spawn_players()

    def generate_map(self, map_name):
        hard_walls = np.load(f'{map_name}/hardwalls.npy')
        vents = np.load(f'{map_name}/vents.npy', allow_pickle=True)
        
        for coord in hard_walls:
            coord[1] = self.height - 1 - coord[1]
            self.new_agent(Wall, tuple(coord))

        self.vents_dict = {}
        self.vents = []

        for connection in vents:
            connection = connection.tolist()
            self.vents_dict[tuple(connection[0])] = tuple(connection[1])

        for coord in self.vents_dict:
            self.new_agent(Vent, tuple(coord))

    def spawn_players(self):
        # generate crewmates
        for _ in range(self.n_crew):
            self.new_agent(Crewmate, (125, 100))

        # generate imposters
        for _ in range(self.n_impo):
            self.new_agent(Imposter, (125, 100))

    def new_agent(self, agent_type, pos):
        '''
        Method that creates a new agent, and adds it to the correct scheduler.
        '''
        agent = agent_type(self.next_id(), self, pos)

        self.grid.place_agent(agent, pos)

        if agent_type == Crewmate or agent_type == Imposter:
            getattr(self, f'schedule_{agent_type.__name__}').add(agent)

        if agent_type == Vent:
            self.vents.append(agent)
        
    def step(self):
        '''
        Method that steps every agent. 
        
        Prevents applying step on new agents by creating a local list.
        '''

        self.schedule_Crewmate.step()
        self.schedule_Imposter.step()

        # self.datacollector.collect(self)

    def play_match(self, step_count=50):
        '''
        Method that runs the model for a specific amount of steps.
        '''
        for i in range(step_count):
            # print(i)
            self.step()

In [137]:
class Crewmate(Agent):
    def __init__(self, unique_id, model, init_pos):
        super().__init__(unique_id, model)
        self.pos = init_pos
        self.path = []
        self.goal = (90, 40)

    def find_goal(self):
        self.path = a_star(self.pos, self.goal, self.model.grid)[0]

    def step(self):
        if self.path == []:
            self.find_goal()
        
        grid = self.model.grid
        neighborhood = grid.get_neighborhood(self.pos, True)

        neighbors = self.model.grid.get_neighbors(self.pos, True)

        if len(neighbors) > 0:
            for neighbor in neighbors:
                if type(neighbor) == Wall and neighbor.pos in neighborhood:
                    neighborhood.remove(neighbor.pos)

        if self.path != []:
            self.model.grid.move_agent(self, self.path.pop())            

class Imposter(Agent):
    def __init__(self, unique_id, model, init_pos):
        super().__init__(unique_id, model)
        self.pos = init_pos
        self.path = []
        self.goals = [(85, 35), (140, 110), (125, 90)]
        self.vents_enabled = True
    
    def find_path(self):
        if len(self.goals) > 0:
            random.shuffle(self.goals)
            goal = self.goals.pop()
            best_path, best_score = a_star(self.pos, goal, self.model.grid)

            if self.vents_enabled:

                for vent in self.model.vents:
                    vent_in = vent.pos_a
                    vent_out = vent.pos_b
                    guess_vent_path = euclidean(self.pos, vent_in) + euclidean(vent_out, goal)

                    if guess_vent_path < best_score:
                        vent_path_in, vent_score_in = a_star(self.pos, vent_in, self.model.grid)
                        vent_path_out, vent_score_out = a_star(vent_out, goal, self.model.grid)
                        vent_path = vent_path_out + vent_path_in
                        vent_score = vent_score_in + vent_score_out
    
                        if vent_score < best_score:
                            best_score = vent_score
                            best_path = vent_path

            self.path = best_path

    def step(self):
        if self.path == []:
            self.find_path()

        grid = self.model.grid
        neighborhood = grid.get_neighborhood(self.pos, True)

        neighbors = self.model.grid.get_neighbors(self.pos, True)

        if len(neighbors) > 0:
            for neighbor in neighbors:
                if type(neighbor) == Wall and neighbor.pos in neighborhood:
                    neighborhood.remove(neighbor.pos)

        if self.path != []:
            self.model.grid.move_agent(self, self.path.pop())               
                        
class Wall(Agent):
    def __init__(self, unique_id, model, pos):
        super().__init__(unique_id, model)
        self.pos = pos

class Obstruction(Agent):
    def __init__(self, unique_id, model, pos):
        super().__init__(unique_id, model)
        self.pos = pos

class Vent(Agent):
    def __init__(self, unique_id, model, pos):
        super().__init__(unique_id, model)
        self.pos_a = pos
        self.pos_b = self.model.vents_dict[pos]

In [138]:
amongUs = AmongUs('the_skeld', 0, 1)

amongUs.play_match()

TypeError: unsupported operand type(s) for -: 'int' and 'list'